# USING LSTM BECAUSE THE PAPER SAID SO
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6121625/

next to explore - time series. focus on multivariate first, before doing multi step

In [1]:
import numpy as np
import pandas as pd

In [2]:
test_features = pd.read_csv("dengue_features_test.csv", index_col=[0,1,2]) 
train_features = pd.read_csv("dengue_features_train.csv", index_col=[0,1,2]) 
train_labels = pd.read_csv("dengue_labels_train.csv", index_col=[0,1,2]) 
submission_format = pd.read_csv("submission_format.csv", index_col=[0,1,2]) 

In [3]:
test_features

week_start_date   ndvi_ne   ndvi_nw   ndvi_se   ndvi_sw  \
city year weekofyear                                                           
sj   2008 18              2008-04-29 -0.018900 -0.018900  0.102729  0.091200   
          19              2008-05-06 -0.018000 -0.012400  0.082043  0.072314   
          20              2008-05-13 -0.001500       NaN  0.151083  0.091529   
          21              2008-05-20       NaN -0.019867  0.124329  0.125686   
          22              2008-05-27  0.056800  0.039833  0.062267  0.075914   
...                              ...       ...       ...       ...       ...   
iq   2013 22              2013-05-28  0.301471  0.380029  0.280629  0.383186   
          23              2013-06-04  0.247600  0.296343  0.285371  0.350357   
          24              2013-06-11  0.238729  0.251029  0.252586  0.249771   
          25              2013-06-18  0.310429  0.302700  0.406614  0.403943   
          26              2013-06-25  0.339467  0.240071  0.356943  0.273600   

                      precipitation_amt_mm  reanalysis_air_temp_k  \
city year weekofyear                                                
sj   2008 18                         78.60             298.492857   
          19                         12.56             298.475714   
          20                          3.66             299.455714   
          21                          0.00             299.690000   
          22                          0.76             299.780000   
...                                    ...                    ...   
iq   2013 22                         41.12             297.774286   
          23                         71.52             297.167143   
          24                         78.96             295.831429   
          25                         39.54             295.778571   
          26                         51.80             297.372857   

                      reanalysis_avg_temp_k  reanalysis_dew_point_temp_k  \
city year weekofyear                                                       
sj   2008 18                     298.550000                   294.527143   
          19                     298.557143                   294.395714   
          20                     299.357143                   295.308571   
          21                     299.728571                   294.402857   
          22                     299.671429                   294.760000   
...                                     ...                          ...   
iq   2013 22                     298.964286                   295.638571   
          23                     298.328571                   295.845714   
          24                     296.607143                   294.894286   
          25                     297.400000                   293.648571   
          26                     299.000000                   294.615714   

                      reanalysis_max_air_temp_k  ...  \
city year weekofyear                             ...   
sj   2008 18                              301.1  ...   
          19                              300.8  ...   
          20                              302.2  ...   
          21                              303.0  ...   
          22                              302.3  ...   
...                                         ...  ...   
iq   2013 22                              305.5  ...   
          23                              306.3  ...   
          24                              304.6  ...   
          25                              305.9  ...   
          26                              307.3  ...   

                      reanalysis_precip_amt_kg_per_m2  \
city year weekofyear                                    
sj   2008 18                                    25.37   
          19                                    21.83   
          20                                     4.12   
          21                                     2.20   
          22          

In [4]:
#check for missing data
train_features.isnull().values.any()

#check for number of missing data
train_features.isnull().sum()

week_start_date                            0
ndvi_ne                                  194
ndvi_nw                                   52
ndvi_se                                   22
ndvi_sw                                   22
precipitation_amt_mm                      13
reanalysis_air_temp_k                     10
reanalysis_avg_temp_k                     10
reanalysis_dew_point_temp_k               10
reanalysis_max_air_temp_k                 10
reanalysis_min_air_temp_k                 10
reanalysis_precip_amt_kg_per_m2           10
reanalysis_relative_humidity_percent      10
reanalysis_sat_precip_amt_mm              13
reanalysis_specific_humidity_g_per_kg     10
reanalysis_tdtr_k                         10
station_avg_temp_c                        43
station_diur_temp_rng_c                   43
station_max_temp_c                        20
station_min_temp_c                        14
station_precip_mm                         22
dtype: int64

In [5]:
#fill with zero if null
train_features.fillna(0, inplace=True)

In [6]:
#check for number of missing data again, should be none
train_features.isnull().sum()

week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_precip_mm                        0
dtype: int64

In [7]:
#check for number of missing data
test_features.isnull().sum()

week_start_date                           0
ndvi_ne                                  43
ndvi_nw                                  11
ndvi_se                                   1
ndvi_sw                                   1
precipitation_amt_mm                      2
reanalysis_air_temp_k                     2
reanalysis_avg_temp_k                     2
reanalysis_dew_point_temp_k               2
reanalysis_max_air_temp_k                 2
reanalysis_min_air_temp_k                 2
reanalysis_precip_amt_kg_per_m2           2
reanalysis_relative_humidity_percent      2
reanalysis_sat_precip_amt_mm              2
reanalysis_specific_humidity_g_per_kg     2
reanalysis_tdtr_k                         2
station_avg_temp_c                       12
station_diur_temp_rng_c                  12
station_max_temp_c                        3
station_min_temp_c                        9
station_precip_mm                         5
dtype: int64

In [8]:
#fill with zero if null
test_features.fillna(0, inplace=True)

In [9]:
#check for number of missing data
test_features.isnull().sum()

week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_precip_mm                        0
dtype: int64

## Model

In [12]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from matplotlib import pyplot

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()
1
2
3
4
5
6
7
8
9
10
11
12
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()